In [ ]:
! dir D:\fork\lipidxplorer-evaluation\190731_benchmark_data_files_infos\190731_mzML_no_zlib

In [ ]:
from pathlib import Path


In [ ]:
from pathlib import Path
p = Path(r'D:\fork\lipidxplorer-evaluation\190731_benchmark_data_files_infos\190731_mzML_no_zlib')
mzmls = list(p.glob("*.mzml"))

In [ ]:
mzmls = list(p.glob("*.mzml"))

In [ ]:
mzmls

In [ ]:
from ms_deisotope import MSFileLoader

In [ ]:
import pandas as pd
import numpy as np
def path2df(path, time_start=0, time_end=float("inf"), ms1_start=0, ms1_end=float("inf"), ms2_start=0, ms2_end=float("inf"), ):
    dfs = []
    with MSFileLoader(str(path)) as r:
        r.get_scan_by_time(time_start / 60)
        r.start_from_scan(r.get_scan_by_time(time_start / 60).id)
        for b in r:
            if not (time_start / 60 < b.precursor.scan_time < time_end / 60):
                continue
            a = b.precursor.arrays
            df = pd.DataFrame({"mz": a.mz, "inty": a.intensity, 'stem': path.stem, "scan_id": b.precursor.scan_id, "filter_string":b.precursor.annotations["filter string"], "precursor_id":np.nan, 'precursor_mz':np.nan})
            df = df[(df.mz.between(ms1_start,ms1_end )) & (df.inty > 0)]
            # df["scan_id"] = b.precursor.scan_id
            # df["filter_string"] = b.precursor.annotations["filter string"]
            # df["precursor_id"] = np.nan
            dfs.append(df)
            for p in b.products:
                if not (time_start / 60 < p.scan_time < time_end /60):
                    continue
                a = p.arrays
                df = pd.DataFrame({"mz": a.mz, "inty": a.intensity, 'stem': path.stem, "scan_id": p.scan_id, "filter_string":p.annotations["filter string"], "precursor_id":b.precursor.scan_id, 'precursor_mz':p.precursor_information.mz})
                df = df[(df.mz.between(ms2_start,ms2_end )) & df.inty > 0]
                # df["scan_id"] = p.scan_id  # TODO  make them categoriacal?
                # df["filter_string"] = p.annotations["filter string"]
                # df["precursor_id"] = b.precursor.scan_id
                dfs.append(df)
    df = pd.concat(dfs)

    return df

In [ ]:
time_range = (33,1080)
ms1_mass_range = (360, 1000)
ms2_mass_range = (150, 1000)

In [ ]:
spectra_dfs = (path2df(mzml,*time_range, *ms1_mass_range, *ms2_mass_range) for mzml in mzmls)

In [ ]:
d = {"a":1,"b":2,'c':3}
d

In [ ]:
d.setdefault(1)
d['f']

In [ ]:
def agg_ms2_spectra_df(df):
    ms2_peaks = df.loc[~df.precursor_id.isna()]
    ms2_agg_peaks =ms2_peaks.groupby([ms2_peaks.precursor_mz.round(2), ms2_peaks.mz.round(2) ]).agg({'mz':['count','mean'], 'inty':['mean'], 'stem':['first']})
    ms2_agg_peaks = ms2_agg_peaks.loc[ms2_agg_peaks.mz['count'] > 1]
    ms2_agg_peaks.columns = ['_'.join(col).strip() for col in ms2_agg_peaks.columns.values]
    return ms2_agg_peaks

In [ ]:
ms2_df = pd.concat((agg_ms2_spectra_df(df) for df in spectra_dfs ))

In [ ]:
for idx, g_df in ms2_df.groupby(level='precursor_mz'):
    for inner_idx, inner_gdf in g_df.groupby(level='mz'):
        mass = inner_gdf.mz_mean.mean()
        dictIntensity = inner_gdf.set_index('stem_first')['inty_mean'].to_dict()
        dictIntensity = defaultdict(lambda: 0,dictIntensity) # to fill with zeroes


In [ ]:
def mass_inty_generator_ms2(ms2_df, target):
    ms2_slice = ms2_df.loc[ms2_df.index.get_level_values(0) == target] #350.20
    for inner_idx, inner_gdf in ms2_slice.groupby(level='mz'):
        mass = inner_gdf.mz_mean.mean()
        dictIntensity = inner_gdf.set_index('stem_first')['inty_mean'].to_dict()
        dictIntensity = defaultdict(lambda: 0, dictIntensity) # to fill with zeroes
        yield mass, dictIntensity

In [ ]:
from collections import defaultdict

In [ ]:
[t for t in mass_inty_generator_ms2(ms2_df, 350.20)]

----

In [ ]:
def mass_inty_generator(ms1_df):
    for idx, df in ms1_df.groupby(ms1_df.index):
        msmass = df.mz_mean.mean()
        smpl = df.set_index('stem_first')['inty_mean'].to_dict()
        smpl = defaultdict(lambda: 0,smpl) # to fill with zeroes
        yield msmass, dictIntensity
    


In [ ]:
gdf_agg = gdf.groupby(gdf.mz.round(2)).agg({'mz':['count','mean'], 'inty':['mean']})
gdf_agg = gdf_agg.loc[gdf_agg.mz['count'] > 1]
gdf_agg

In [ ]:
df = pd.concat((path2df(mzml,*time_range, *ms1_mass_range, *ms2_mass_range) for mzml in mzmls))


In [ ]:
df

In [ ]:
df[~df.precursor_id.isna()]

In [ ]:
for id, gdf in df.groupby('stem'):
    break

In [ ]:
# NOTE: LX1 misses some scans, seems and lx2 does have it see 
gdf[~gdf.precursor_id.isna()].scan_id.unique().shape

In [ ]:
#https://stackoverflow.com/questions/39475968/aligning-pandas-dataframes-that-dont-have-a-common-index?noredirect=1&lq=1
# https://stackoverflow.com/questions/62482609/find-nearest-index-in-one-dataframe-to-another


In [ ]:

# gdf[gdf.precursor_id.isna()].groupby(gdf[gdf.precursor_id.isna()].mz.round(2)).agg({'mz':['count','mean', 'std'], 'inty':['mean', 'std']})
masterscan = 
masterscandf[df.precursor_id.isna()].groupby(df[df.precursor_id.isna()].mz.round(2)).agg({'mz':['count','mean', 'std'], 'inty':['mean', 'std']}) #.unstack()

In [ ]:
masterscan[masterscan.mz['count'] > 1]

In [ ]:
masterscan_ms2 = df[~df.precursor_id.isna()].groupby(df[~df.precursor_id.isna()].mz.round(2)).agg({'mz':['count','mean', 'std'], 'inty':['mean', 'std']})


In [ ]:
( 6*60 ) * 30 =390
vs


In [ ]:
masterscan_ms2

In [ ]:
masterscan_ms2[masterscan_ms2.mz['count'] > masterscan_ms2.mz['count'].median()]